# EIGN: A universal model for edge-level problems

Here, we will show you how to use the EIGN model for edge-level problems on a dummy graph.

In [1]:
from eign import EIGNLaplacianConv, EIGNLaplacianWithNodeTransformationConv
import torch
import torch.nn as nn

### Create a dummy graph

Let's create a dummy graph with some directed and undirected edges. We assign each edge with features that have an inherent direction (signed or orientation equivariant features) and some features that do not have an inherent direction (unsigned or orientation invariant features).

The signed features are represented with respect to some arbitrary edge orientation through their sign. The edge orientation is encoded through the `edge_index`: If it contains an edge `(u, v)`, signed features are represented relative to the orientation `u -> v`.


In [ ]:
num_signed_features = 3
num_unsigned_features = 6

# Represent the graph
edge_index = (
    torch.tensor(
        [
            [0, 1],
            [1, 2],
            [2, 3],
            [2, 4],
            [3, 5],
            [5, 0],
            [5, 2],
        ]
    )
    .t()
    .contiguous()
)
edge_is_directed = torch.tensor([0, 0, 0, 0, 0, 1, 1], dtype=torch.bool)

dummy_features_signed = torch.randn(edge_index.size(1), num_signed_features)
dummy_features_unsigned = torch.randn(edge_index.size(1), num_unsigned_features)


### Create the basic EIGN model

We can create an instance of EIGN easily using the `eign` package. It produces both signed outputs (w.r.t. edge orientation) and unsigned outputs.

In [ ]:
model = EIGNLaplacianConv(
    in_channels_signed=num_signed_features,
    in_channels_unsigned=num_unsigned_features,
    hidden_channels_signed=32,
    hidden_channels_unsigned=32,
    out_channels_signed=1,
    out_channels_unsigned=5,
    num_blocks=4,
    q=1 / edge_index.size(1),
).eval()

In [4]:
with torch.no_grad():
    outputs = model(
        x_signed=dummy_features_signed,
        x_unsigned=dummy_features_unsigned,
        edge_index=edge_index,
        is_directed=edge_is_directed,
    )
outputs.signed, outputs.unsigned

/nfs/homedirs/fuchsgru/magnetic_edge_gnn/magnetic_edge_gnn_public/eign/laplacian.py:88: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at ../aten/src/ATen/SparseCsrTensorImpl.cpp:53.)
  laplacian = (


(tensor([[ 0.1007],
         [-0.0815],
         [ 0.0641],
         [ 0.1016],
         [ 0.0190],
         [-0.0268],
         [-0.0208]]),
 tensor([[ 0.0242,  0.0358, -0.0247, -0.0139,  0.0613],
         [-0.0019,  0.0054, -0.0602, -0.0166, -0.0144],
         [ 0.0164, -0.0019, -0.0819, -0.0421,  0.0036],
         [-0.0058, -0.0603, -0.0896,  0.0203,  0.0134],
         [-0.0644,  0.0159,  0.1028,  0.0721, -0.0574],
         [-0.0836, -0.0815,  0.0123,  0.0555, -0.0432],
         [-0.0831, -0.0596, -0.0013,  0.0864, -0.1022]]))

### Verify Orientation Equivariance and Invariance

If we change the orientation of *undirected* edges, EIGN's outputs do not change.

In [ ]:
# Flip the orientation of edges 2 and 4
orientation_flipped = torch.zeros(edge_index.size(1), dtype=torch.bool)
orientation_flipped[2] = 1
orientation_flipped[4] = 1

edge_index_flipped = edge_index.clone()
edge_index_flipped[:, orientation_flipped] = edge_index_flipped.flip(0)[
    :, orientation_flipped
]

# Represent signed features in this new orientation
dummy_features_signed_flipped = dummy_features_signed.clone()
dummy_features_signed_flipped[orientation_flipped] = -dummy_features_signed_flipped[
    orientation_flipped
]

with torch.no_grad():
    outputs_flipped = model(
        x_signed=dummy_features_signed_flipped,
        x_unsigned=dummy_features_unsigned,
        edge_index=edge_index_flipped,
        is_directed=edge_is_directed,
    )


Unsigned signals which are not represented relative to an orientation remain unchanged.

In [ ]:
outputs_flipped.unsigned - outputs.unsigned

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

Signed outputs also are not changed. However, since we changed the orientation of some edges, the sign of the corresponding signed signals have flipped. Intuitively, we have changed the basis in which signed signals are represented. This change is reflected through the sign in both EIGN's inputs and outputs.

In [ ]:
# The signed outputs did not change as well, but are represented w.r.t. the new orientation
outputs_flipped_signed_old_orientation = outputs_flipped.signed.clone()
# If we reorient them back into the old orientation ...
outputs_flipped_signed_old_orientation[
    orientation_flipped
] = -outputs_flipped_signed_old_orientation[orientation_flipped]
# ... we get the same values
outputs_flipped_signed_old_orientation - outputs.signed

tensor([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]])

However, if we change the orientation (=direction) of a *directed* edge, EIGN is sensitive to that and outputs different signed and unsigned features. It can therefore learn to model the direction of directed edges and still represent signed and unsigned edge signals.

In [ ]:
# Change the orientation of a directed edge
orientation_flipped_directed = torch.zeros(edge_index.size(1), dtype=torch.bool)
orientation_flipped_directed[5] = 1

edge_index_flipped_directed = edge_index.clone()
edge_index_flipped_directed[:, orientation_flipped_directed] = (
    edge_index_flipped_directed.flip(0)[:, orientation_flipped_directed]
)
edge_index

tensor([[0, 1, 2, 2, 3, 5, 5],
        [1, 2, 3, 4, 5, 0, 2]])

In [ ]:
# Represent signed features in this new orientation

dummy_features_signed_flipped_directed = dummy_features_signed.clone()
dummy_features_signed_flipped_directed[
    orientation_flipped_directed
] = -dummy_features_signed_flipped_directed[orientation_flipped_directed]

with torch.no_grad():
    outputs_flipped_directed = model(
        x_signed=dummy_features_signed_flipped_directed,
        x_unsigned=dummy_features_unsigned,
        edge_index=edge_index_flipped_directed,
        is_directed=edge_is_directed,
    )


In [ ]:
# Now, the change in orientation (i.e. direction) of the edge breaks orientation invariance ...
outputs.unsigned - outputs_flipped_directed.unsigned

tensor([[ 0.0324, -0.0088,  0.0055, -0.0100,  0.0433],
        [ 0.0167, -0.0103, -0.0142,  0.0043,  0.0168],
        [ 0.0024, -0.0097, -0.0137, -0.0076, -0.0008],
        [ 0.0036, -0.0034, -0.0163, -0.0099,  0.0057],
        [-0.0236,  0.0302,  0.0549,  0.0346, -0.0076],
        [ 0.0355, -0.0349, -0.0252, -0.0408,  0.0131],
        [-0.0321, -0.0141,  0.0447,  0.0194, -0.0304]])

In [ ]:
# ... and also orientation equivariance (w.r.t the old orientation)
outputs_flipped_directed_old_orientation = outputs_flipped_directed.signed.clone()
outputs_flipped_directed_old_orientation[
    orientation_flipped_directed
] = -outputs_flipped_directed_old_orientation[orientation_flipped_directed]
outputs_flipped_directed_old_orientation - outputs.signed


tensor([[-0.0116],
        [-0.0009],
        [ 0.0024],
        [ 0.0063],
        [-0.0602],
        [ 0.0446],
        [ 0.0689]])

### Different variations of EIGN

EIGN's Magnetic Laplacian operators also allow representing edge signals at the node level which further increases its expressivity (see our [paper](https://arxiv.org/pdf/2410.16935) for further reading).

In [ ]:
model = EIGNLaplacianWithNodeTransformationConv(
    in_channels_signed=num_signed_features,
    in_channels_unsigned=num_unsigned_features,
    hidden_channels_signed=32,
    hidden_channels_unsigned=32,
    out_channels_signed=1,
    out_channels_unsigned=5,
    num_blocks=4,
    q=1 / edge_index.size(1),
).eval()

You can even define your own (learnable) mapping to transform node features that are induced by the Magnetic Edge Laplacian's boundary operators. By default, a `ReLU` is used to introduce some nonlinearity. You can use whatever you want, even a node-level GNN is possible here.

In [ ]:
def initialize_node_feature_transformation(
    num_channels_in: int, num_channels_out: int
) -> nn.Module:
    return nn.Sequential(
        nn.ReLU(),
        nn.Linear(num_channels_in, num_channels_out),
        nn.ReLU(),
        nn.Linear(num_channels_out, num_channels_out),
    )


model = EIGNLaplacianWithNodeTransformationConv(
    in_channels_signed=num_signed_features,
    in_channels_unsigned=num_unsigned_features,
    hidden_channels_signed=32,
    hidden_channels_unsigned=32,
    out_channels_signed=1,
    out_channels_unsigned=5,
    num_blocks=4,
    q=1 / edge_index.size(1),
    initialize_node_feature_transformation=initialize_node_feature_transformation,
).eval()